# Encoder 출력(hs) -> 가중치(a) -> 가중합

In [ ]:
import numpy as np

T,H = 5,4
hs = np.random.randn(T, H)
a = np.array([0.8,0.1,0.03,0.05,0.02])

ar = a.reshape(5,1).repeat(4,axis=1)
print(ar.shape)
# (5,4)
"""
구현 효율만 생각하면 repeat() 보다는 넘파이의 브로드캐스트 이용.
하지만 다차원 배열의 원소가 복사(반복)되고 있다는 점이
우리의 눈에는 보이지 않으니 주의해야함.
이 작업은 계산그래프 상으로는 Repeat 노드에 해당

따라서 역전파 때는 repeat 노드의 역전파를 수행해야함.
"""

t = hs*ar
print(t.shape)
# (5,4) -> 여기서 0번째 축을 사라지게 하니까 

c = np.sum(t, axis=0)
print(c.shape)
# (4,) -> 이렇게 나온 것

"""
x의 형상이 (X,Y,Z)일때
np.sum(x,axis=1)을 실행하면
출력 형상은 (X,Z)
= 1번쨰 축이 사라진다
"""

### 미니배치 처리용 가중합

In [ ]:
N, T, H = 10, 5, 4
hs = np.random.randn(N,T,H)
a = np.random.randn(N,T)
ar = a.shape(N,T,1).repeat(H, axis=2) #
# ar = a.shape(N,T,1) : broadcast

t = hs * ar
print(t.shape)

c = np.sum(t, axis=1) #
print(c.shape)

"""
미니배치 구현은 이전과 거의 같습니다. 
배열의 형상을 잘 살펴보면 repeat()와 sum()에서
어느 차원(축)을 지정해야하는지 곧 알 수 있을 것입니다.
"""

# Weight Sum 계층
: 맥락 벡터(c)를 구하는 계층

In [ ]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], [] # 학습하는 매개변수 없으므로 params 비워두기
        self.cache = None

    def forward(self, hs, a):
        N, T, H = hs.shape

        ar = a.reshape(N,T,1).repeat(H, axis = 2)
        t = hs*ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c
    
    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape

        dt = dc.reshape(N,1,H).repeat(T,axis=1) # sum의 역전파
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2) #repeat의 역전파

        return dhs, da